# Table of Contents
* [Investigating programatically recovering hierarchy](#Investigating-programatically-recovering-hierarchy)
	* [Dataframe stuff](#Dataframe-stuff)
* [End](#End)


In [1]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, OrderedDict
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import json
import glob
import pprint

In [3]:
import pdfextraction.merge as me
import pdfextraction.unmerge as um
import pdfextraction.amt_boto_modules as amt
import pdfextraction.hierarchy_exctraction as hext

# Investigating programatically recovering hierarchy

In [4]:
base_path = 'ai2-vision-turk-data/textbook-annotation-test/labeled-questions/'
test_path = 'ai2-vision-turk-data/textbook-annotation-test/hierarchy-ext-test/'

In [45]:
question_pages_df = pd.read_pickle('pages_w_questions.pkl')
question_pages = question_pages_df['page'].tolist()
to_review = pd.read_pickle('rev_seq.pkl').tolist()

book_breakdowns, page_ranges = amt.load_book_info()

verified_pages = to_review
verified_pages_minus_bad = [page for page in verified_pages if page not in bad_pages.tolist()]
print len(verified_pages)
print len(verified_pages_minus_bad)

856
468


## Basic structure

In [356]:
single_page = ['Daily_Science_Grade_2_Evan_Moor_33.jpeg']

In [244]:
# accuracy_total, errors_total, box_totals, ind_page_results = predict_and_verify_groups(single_page)
accuracy_total, errors_total, box_totals, ind_page_results = hext.predict_and_verify_groups(verified_pages_minus_bad, base_path)

print errors_total
print box_totals
print accuracy_total

822
11602
0.929150146526


In [248]:
for page_res in ind_page_results:
    hext.write_predicted_groups(page_res, base_path, test_path)

In [42]:
problem_pages = [page[0] for page in ind_page_results if page[2] < 1.0]
problem_pagess = pd.Series(problem_pages)
# problem_pagess.to_pickle('problem_pages.pkl')

In [265]:
page_n = 0

In [281]:
cur_page = problem_pages[page_n:page_n + 1]
accuracy_total, errors_total, box_totals, ind_page_results = predict_and_verify_groups(cur_page)
page_n += 1
print ''
print cur_page
print '{} {}'.format('page', page_n)
print '{} {}'.format('number wrong=', errors_total)
print '{} {}'.format('out of', box_totals)
print '{} {}'.format('accuracy', accuracy_total)


actual 1
predicted 1
A. Write true or false.


actual 1
predicted 1
1. A conductor allows electric current to


actual 1
predicted 1
flow through it easily


actual 1
predicted 1
2. Electrical energy can make heat energy.


actual 1
predicted 1
3. Resistors convert less energy into heat


actual 1
predicted 1
than conductors do


actual 1
predicted 1
4. All metals conduct electricity


actual 1
predicted 1
equally we


actual 2
predicted 2
B. What is the main difference between a conductor and a resistor?


actual 2
predicted 3
C. How did resistors help the people who invented toasters?


actual 2
predicted 3
Explain your answer


[u'Daily_Science_Grade_4_Evan_Moor_131.jpeg']
page 9
number wrong= 2
out of 22
accuracy 0.909090909091


## Question-type specific structure

In [297]:
import pprint

In [232]:
import re
import string

In [342]:
hext.assemble_questions(single_page, test_path)

miss
A. Fill in the bubble next to the correct answer.

miss
B. Which one would win a race, a sound wave in the air or a sound wave

miss
in the water? Why? Write a sentence to answer the question



{u'full_Q_1': {'answer_choices': {u'answer_choice_A': {'possible_answer': {u'box_id': u'Q10',
     u'contents': u'A Neither of them lets sound waves travel',
     u'group_n': 1,
     u'rectangle': [[224, 915], [1050, 953]]},
    'structural_label': u'A'},
   u'answer_choice_B': {'possible_answer': {u'box_id': u'Q11',
     u'contents': u'B Both have molecules',
     u'group_n': 1,
     u'rectangle': [[225, 976], [685, 1016]]},
    'structural_label': u'B'},
   u'answer_choice_C': {'possible_answer': {u'box_id': u'Q12',
     u'contents': u'C Our ears can hear sound we in both',
     u'group_n': 1,
     u'rectangle': [[225, 1040], [998, 1081]]},
    'structural_label': u'C'}},
  'asks': {'Qc1': {u'box_id': u'Q9',
    u'contents': u'1. How are air and liquid alike?',
    u'group_n': 1,
    u'rectangle': [[175, 850], [767, 900]]}},
  'category': u'Multiple Choice',
  'question_id': u'full_Q_1',
  'structural_id': u'1.'},
 u'full_Q_2': {'answer_choices': {u'answer_choice_A': {'possible_answe

In [224]:
#print vertically_ordered_question
# qdf = pd.DataFrame(vertically_ordered_question)

# Dataframe stuff

In [118]:
# single_page_df = pd.DataFrame(vertically_ordered_question)

In [119]:
# single_page_df['rectangle'].iloc[0]

In [24]:
qa_df = pd.read_pickle('store_hit_results_metadata/question_anno/group_latest_combined/consensus_df.pkl')

In [25]:
bad_pages = pd.unique(qa_df.query('category == "No Consensus" or group_n == "No Consensus"')['page'])

In [26]:
consensus_only_df = qa_df[~qa_df['page'].isin(bad_pages)]

In [27]:
pd.unique(consensus_only_df['page']).shape

(468,)

In [28]:
consensus_only_df.head(4)

,page,box_id,category,hit_id,group_n
20,Daily_Science_Grade_1_Evan_Moor_100.jpeg,Q11,Fill-in-the-Blank,3VDVA3ILIDG3ODEZECC2QGUXV88G1P,1
21,Daily_Science_Grade_1_Evan_Moor_100.jpeg,Q13,Fill-in-the-Blank,3VDVA3ILIDG3ODEZECC2QGUXV88G1P,1
22,Daily_Science_Grade_1_Evan_Moor_100.jpeg,Q14,Fill-in-the-Blank,3VDVA3ILIDG3ODEZECC2QGUXV88G1P,1
23,Daily_Science_Grade_1_Evan_Moor_100.jpeg,Q15,Fill-in-the-Blank,3VDVA3ILIDG3ODEZECC2QGUXV88G1P,1


# End

In [363]:
# vertically_ordered_question = sorted(q_series.values(), key=lambda x: x['rectangle'][0][1])

# vertically_ordered_question_feat = [make_box_row(box, idx) for idx, box in enumerate(vertically_ordered_question)]

# boxes_w_predicts = assign_group_numbers(vertically_ordered_question_feat) 

In [310]:
t0 = 'Read each question. Fill in the bubble next to the correct answer.'
t1 = 'A distance'
t2 = 'OB balls'
t3 = 'Do push'
t4 = 'OD on a mountain'
t5 = 'CO Flowers might die out.'
re_pattern = re.compile('([A-Z])(?:[a-z]?){1}\s')

print re.findall(re_pattern, t0)
print re.findall(re_pattern, t1)
print re.findall(re_pattern, t2)
print re.findall(re_pattern, t3)
print re.findall(re_pattern, t4)
print re.findall(re_pattern, t5)